In [1]:
from keras import layers
from keras import initializers

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [32]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
def convolution_block(X_IN, filter_size, number_of_filters, stage, block, stride=2):
    name_fmt = 'convolution_block_layer_{}' + 'stage_{}_block_{}'.format(stage, block) + '_branch_{}'
    F1, F2, F3 = number_of_filters
    
    # 1x1 convolutions to change the number of filters and reduce the dimensions of the input
    X = layers.Conv2D(F1, (1, 1), 
                      strides=(stride, stride),
                      padding='same',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'a'))(X_IN)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'a'))(X)
    X = layers.Activation('relu')(X)
    
    # filter_size x filter_size convolution to change the number of filters and change the dimensions
    X = layers.Conv2D(F2, (filter_size, filter_size), 
                      strides=(1, 1),
                      padding='same',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'b'))(X)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'b'))(X)
    X = layers.Activation('relu')(X)
    
    # 1 x 1 convolution to change the number of filters
    X = layers.Conv2D(F3, (1, 1), 
                      strides=(1, 1),
                      padding='valid',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'c'))(X)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'c'))(X)
    
    X_shortcut = layers.Conv2D(F3, (1, 1), 
                               strides=(stride, stride), 
                               padding='valid', 
                               kernel_initializer=initializers.glorot_uniform(seed=0),
                               name=name_fmt.format('conv_short', '1'))(X_IN)
    X_shortcut = layers.BatchNormalization(axis=3, name=name_fmt.format('bn_short', '1'))(X_shortcut)
    
    X = layers.Add()([X, X_shortcut])
    return layers.Activation('relu')(X)

In [4]:
def identity_block(X_IN, filter_size, number_of_filters, stage, block):
    name_fmt = 'identity_block_layer_{}' + 'stage_{}_block_{}'.format(stage, block) + '_branch_{}'
    F1, F2, F3 = number_of_filters
    
    X = layers.Conv2D(F1, (1, 1), strides=(1, 1), padding='valid',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'a'))(X_IN)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'a'))(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(F2, (filter_size, filter_size), strides=(1, 1), padding='same',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'b'))(X)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'b'))(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(F3, (1, 1), strides=(1, 1), padding='valid',
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name=name_fmt.format('conv', 'c'))(X)
    X = layers.BatchNormalization(axis=3, name=name_fmt.format('bn', 'c'))(X)
    
    X = layers.Add()([X, X_IN])
    return layers.Activation('relu')(X)
    
    

In [5]:
def residual_stage(X_IN, filter_size, number_of_filters, stage, number_of_blocks=1):
    
    X = convolution_block(X_IN, filter_size, number_of_filters, stage, 0)
    number_of_blocks -= 1
    
    for i in range(number_of_blocks):
        X = identity_block(X, filter_size, number_of_filters, stage, i + 1)
    
    return X

In [25]:
from keras.models import Model

def Resnet50(input_shape, classes):
    X_input = layers.Input(input_shape)
    X = layers.ZeroPadding2D((3, 3))(X_input)
    X = layers.Conv2D(64, (7, 7), strides=(2, 2), 
                      padding='same', 
                      kernel_initializer=initializers.glorot_uniform(seed=0),
                      name='conv_stage_1')(X)
    X = layers.BatchNormalization(axis=3, name='bn_stage_1')(X)
    print(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
    print(X)
    
    X = residual_stage(X, 3, [64, 64, 256], 2, number_of_blocks=3)
    print(X)
    X = residual_stage(X, 3, [128, 128, 512], 3, number_of_blocks=4)
    print(X)
    X = residual_stage(X, 3, [256, 256, 1024], 4, number_of_blocks=3)
    print(X)
    
    X = layers.AveragePooling2D()(X)
    print(X)
    
    X = layers.Flatten()(X)
    X = layers.Dense(classes, activation='softmax',
                     kernel_initializer=initializers.glorot_uniform(seed=0),
                     name='fully_connected_5')(X)
    return Model(inputs=X_input, outputs=X, name='Resnet50')

In [26]:
x_train.shape

(50000, 32, 32, 3)

In [28]:
resnet50 = Resnet50((32, 32, 3), 10)

Tensor("bn_stage_1_7/cond/Merge:0", shape=(?, 19, 19, 64), dtype=float32)
Tensor("max_pooling2d_8/MaxPool:0", shape=(?, 9, 9, 64), dtype=float32)
Tensor("activation_255/Relu:0", shape=(?, 5, 5, 256), dtype=float32)
Tensor("activation_267/Relu:0", shape=(?, 3, 3, 512), dtype=float32)
Tensor("activation_276/Relu:0", shape=(?, 2, 2, 1024), dtype=float32)
Tensor("average_pooling2d_8/AvgPool:0", shape=(?, 1, 1, 1024), dtype=float32)


In [30]:
resnet50.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 38, 38, 3)    0           input_8[0][0]                    
__________________________________________________________________________________________________
conv_stage_1 (Conv2D)           (None, 19, 19, 64)   9472        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
bn_stage_1 (BatchNormalization) (None, 19, 19, 64)   256         conv_stage_1[0][0]               
__________________________________________________________________________________________________
max_poolin

In [33]:
import keras
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [36]:
resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet50.fit(x_train, y_train, epochs = 1, batch_size = 128)

Epoch 1/1
20608/50000 [===========>..................] - ETA: 3:46 - loss: 2.0549 - acc: 0.3472

KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-2-2c530a9741f4>, line 1)